In [ ]:
from mainnet_launch.pages.gas_costs.gas_costs import _fetch_gas_costs_df, fetch_tokemak_address_constants_dfs
from mainnet_launch.database.schema.postgres_operations import TableSelector, merge_tables_as_df
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.constants import ETH_CHAIN, ChainData

from mainnet_launch.database.schema.full import Transactions, Blocks, ChainlinkGasCosts
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

In [ ]:
def _fetch_chainlink_keeper_network_transactions(chain: ChainData, chainlink_keepers_df: pd.DataFrame) -> pd.DataFrame:
    chainlink_gas_costs_df = merge_tables_as_df(
        [
            TableSelector(ChainlinkGasCosts, select_fields=[ChainlinkGasCosts.chainlink_topic_id]),
            TableSelector(
                Transactions,
                join_on=ChainlinkGasCosts.tx_hash == Transactions.tx_hash,
            ),
            TableSelector(Blocks, select_fields=Blocks.datetime, join_on=Transactions.block == Blocks.block),
        ],
        where_clause=Blocks.chain_id == chain.chain_id,
    )
    chainlink_gas_costs_df["gas_cost_in_eth"] = (
        chainlink_gas_costs_df["gas_cost_in_eth"] * 1.2
    )  # chainlink charges a 20% premium

    keeper_network_topic_ids_to_name = (
        chainlink_keepers_df[chainlink_keepers_df["chain_id"] == chain.chain_id].set_index("id")["name"].to_dict()
    )

    chainlink_gas_costs_df["category"] = "keeper_network"
    chainlink_gas_costs_df["label"] = chainlink_gas_costs_df["chainlink_topic_id"].map(keeper_network_topic_ids_to_name)
    return chainlink_gas_costs_df[
        ["datetime", "label", "category", "effective_gas_price", "gas_used", "gas_cost_in_eth"]
    ]


def _fetch_eoa_transactions(
    chain: ChainData, deployers_df: pd.DataFrame, service_accounts_df: pd.DataFrame
) -> pd.DataFrame:
    address_to_name = service_accounts_df.set_index("address")["name"].to_dict()
    address_to_name[deployers_df["deployer"].values[0]] = "deployer"

    address_to_category = service_accounts_df.set_index("address")["type"].to_dict()
    address_to_category[deployers_df["deployer"].values[0]] = "deployer"

    eoa_tx_df = merge_tables_as_df(
        [
            TableSelector(
                Transactions,
                row_filter=Transactions.from_address.in_(list(address_to_name.keys())),
            ),
            TableSelector(Blocks, select_fields=Blocks.datetime, join_on=Transactions.block == Blocks.block),
        ],
        where_clause=Blocks.chain_id == chain.chain_id,
    )

    eoa_tx_df["label"] = eoa_tx_df["from_address"].map(address_to_name)
    eoa_tx_df["category"] = eoa_tx_df["from_address"].map(address_to_category)

    return eoa_tx_df[["datetime", "label", "category", "effective_gas_price", "gas_used", "gas_cost_in_eth"]]


def fetch_gas_costs_transactions_df() -> pd.DataFrame:
    dfs = []
    deployers_df, chainlink_keepers_df, service_accounts_df = fetch_tokemak_address_constants_dfs()
    for chain in [ETH_CHAIN]:
        chainlink_gas_costs_df = _fetch_chainlink_keeper_network_transactions(chain, chainlink_keepers_df)
        eoa_tx_df = _fetch_eoa_transactions(chain, deployers_df, service_accounts_df)

        full_tx_df = pd.concat([chainlink_gas_costs_df, eoa_tx_df], ignore_index=True)
        dfs.append(full_tx_df)

    df = pd.concat([chainlink_gas_costs_df, eoa_tx_df], ignore_index=True)

    df["label"] = df["label"] + " (" + df["category"] + ")"
    return df


df = fetch_gas_costs_transactions_df()
df

,datetime,label,category,effective_gas_price,gas_used,gas_cost_in_eth
0,2025-07-03 06:44:59+00:00,Incentive Pricing (keeper_network),keeper_network,498225940,214745,0.000128
1,2025-02-10 04:12:11+00:00,Calculators (keeper_network),keeper_network,1365718489,534860,0.000877
2,2025-05-30 12:00:59+00:00,Calculators (keeper_network),keeper_network,3357016839,1818928,0.007327
3,2024-11-30 12:00:23+00:00,Calculators (keeper_network),keeper_network,7659989110,1703275,0.015656
4,2025-01-14 04:22:47+00:00,Calculators (keeper_network),keeper_network,2841546074,414842,0.001415
...,...,...,...,...,...,...
51045,2025-07-15 00:07:47+00:00,Liquidator (rewards),rewards,919485883,2537512,0.002333
51046,2025-07-15 00:07:23+00:00,Liquidator (rewards),rewards,3950519048,1873497,0.007401
51047,2025-07-14 23:53:59+00:00,Custom Oracle Executor (pricing),pricing,2571813534,40641,0.000105
51048,2025-07-15 00:07:11+00:00,Liquidator (rewards),rewards,3945998248,2009384,0.007929


In [ ]:
# df.groupby(['label', 'category']).agg({'effective_gas_price': ['mean', 'median'], 'gas_used': ['sum']}).reset_index().sort_values(('gas_used', 'sum')
# )

In [ ]:
break

In [ ]:
import streamlit as st

for col in ["category", "label"]:
    st.subheader(f"Gas Costs in ETH and Gas Used by Expense {col}")
    for value_col in ["gas_cost_in_eth", "gas_used"]:
        daily_sum_df = (
            df.groupby([col, "datetime"])[value_col]
            .sum()
            .reset_index()
            .pivot(index="datetime", columns=col, values=value_col)
            .fillna(0)
            .resample("1D")
            .sum()
        )

        st.subheader(f"{value_col} by {col}")
        for n_days in [1, 7, 30]:
            fig = px.bar(
                daily_sum_df.rolling(window=n_days).sum(),
                title=f"{n_days}-day rolling sum Mainnet {value_col}",
            )
            fig.update_yaxes(title_text=value_col)
            st.plotly_chart(fig, use_container_width=True)

In [ ]:
# Ensure datetime column is proper and extract the date
df["date"] = pd.to_datetime(df["datetime"]).dt.date

# Create weighted value column
df["weighted_value"] = df["effective_gas_price"] * df["gas_used"] / 1e9

# Group and compute sum of weights and weighted values
grouped = df.groupby(["date", "category"]).agg(weighted_sum=("weighted_value", "sum"), total_weight=("gas_used", "sum"))

# Compute weighted average
grouped["weighted_avg_effective_gas_price"] = grouped["weighted_sum"] / grouped["total_weight"]

# Optionally drop intermediate columns
grouped = (
    grouped.drop(columns=["weighted_sum", "total_weight"])
    .reset_index()
    .pivot(index="date", columns="category", values="weighted_avg_effective_gas_price")
)

px.scatter(
    grouped,
    title="Weighted Average Effective Gas Price by Date and EOA Address",
    labels={"value": "Weighted Average Effective Gas Price", "date": "Date"},
).update_yaxes(title_text="Effective Gas Price (Gwei)").show()

In [ ]:
px.scatter(
    grouped,
    title="Weighted Average Effective Gas Price by Date and EOA Address",
    labels={"value": "Weighted Average Effective Gas Price", "date": "Date"},
).update_yaxes(title_text="Effective Gas Price (Gwei)").show()

In [ ]:
# Ensure datetime column is proper and extract the date
df["date"] = pd.to_datetime(df["datetime"]).dt.date

gas_used_by_day = df.groupby(["date", "category"]).agg(total_gas_used=("gas_used", "sum")).reset_index()

# # Compute weighted average}|

# Optionally drop intermediate columns
gas_used_by_day = gas_used_by_day.pivot(index="date", columns="category", values="total_gas_used")

px.bar(
    gas_used_by_day,
    title="Total gas used by category",
).update_yaxes(title_text="Total gas used").show()

In [ ]:
df["date"].value_counts()

In [ ]:
from datetime import datetime

df[df["date"] == datetime.date(2025, 6, 12)]

In [ ]:
px.line(
    gas_used_by_day.rolling(window=7).sum(),
    title="Total gas used by category",
).update_yaxes(title_text="Total gas used").show()

In [ ]:
px.box(grouped.
    title="Weighted Average Effective Gas Price by Date and EOA Address",
    labels={"value": "Weighted Average Effective Gas Price", "date": "Date"},
).update_yaxes(title_text="Effective Gas Price (Gwei)").show()


In [ ]:
px.scatter(
    grouped,
    title="Weighted Average Effective Gas Price by Date and EOA Address",
    labels={"value": "Weighted Average Effective Gas Price", "date": "Date"},
).update_yaxes(title_text="Effective Gas Price (Gwei)").show()